In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb
os.chdir(main)
%run stdPlotting.ipynb
os.chdir(d['py'])
from gmsPython import nestingTree
from valueShares import nestedShares_noOutputs, nestedShares
from mCGE import NCP_CGE as CGE

# CGE model

*Load data:*

In [2]:
t0 = 2019 # set baseline year
name = 'vSimple' # specify name of the CGE version we are currently working on
db = GpyDB(os.path.join(d['data'], f'{name}_{t0}'), name = '_'.join([name, str(t0), 'calibration']), ws = d['work']) # load and rename database 
dbIO = db.copy() # create copy without adjustments made along the way
ws = db.ws # run everything from the same ws

Initialize model:

In [3]:
M = CGE('_'.join([name, str(t0), 'CGE','calibration']), database = db, 
        opt = False, solveWithNLP = False) # don't use optimization in the calibration stage, use NLP solver though

## 1. Add modules

### 1.1. Production

The production module is initialized with a nesting tree and value shares to identify the $\mu$-share parameters of the nesting structure.

In [4]:
nest = nestingTree.Tree('P', tree = db('nestProduction').to_list(), f = 'CES')
nestAgg = nestingTree.AggTree(name = 'P', trees = {t.name: t for t in [nest]}, ws = ws)
nestAgg(namespace = {str(n)+'_input':n for n in db('n')})

In [5]:
v = nestedShares(nestAgg, ws = ws)
db_p = v(dbIO)

Use share parameters and values to get initial values for intermediates:

In [6]:
db.aom(db_p('mu').xs(t0), name = 'mu')
db.aom(adj.rc_pd(db_p('vD'), nest.get('int')).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

Use static user cost as initial guess for price on durables (if no other has been provided):

In [7]:
db.aom(db('pD_dur'), name = 'pD', priority = 'first') # pD_dur includes an estimate of the long-run user-cost of capital

Next, we initialize the production module:

In [8]:
M.addDynamicProdModule(nestAgg, partial = True, mtype = 'DynamicNCES',initFromGms = 'initFirmValue', taxInstr = 'tauLump', 
                       addEmissions = True, abateCosts =True, addAdjCosts = True)

Now, add investment sector:

In [9]:
nest = nestingTree.AggTree(name = 'I', trees = {'I': nestingTree.Tree('I', tree = db('nestInvestment').to_list(), f = 'CES')}, ws = ws)
nest(namespace = {str(n)+'_input':n for n in db('n')})

*Define share parameters:*

In [10]:
v = nestedShares(nest, ws = ws)
db_vs = v(dbIO) 

*Use share parameters and values to get initial values for intermediates:*

In [11]:
db.aom(db_vs('mu').xs(t0), name = 'mu') 
db.aom(adj.rc_pd(db_vs('vD'), nest.get('int')).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

*Add module as production type to CGE:*

In [12]:
M.addInvestmentModule(nest, partial = True, mtype = 'NestedCES')

### 1.2. Households

*Nesting tree:*

In [13]:
cesNest = nestingTree.Tree('HH', tree = db('nestHH').to_list(), f = 'CES')
nest = nestingTree.AggTree(name = 'HH', trees = {t.name: t for t in [cesNest]}, ws = ws)
nest(namespace = {str(n)+'_input': n for n in db('n')})

Value shares:

In [14]:
v = nestedShares_noOutputs(nest, ws = ws)
db_vs = v(dbIO) # one-liner that returns solution database

Add to main database:

In [15]:
db.aom(db_vs('mu').xs(t0), name = 'mu') 
db.aom(adj.rc_pd(db_vs('vD'), nest.get('int').union(nest.get('output'))).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

Add as consumer module:

In [16]:
M.addConsumerModule(nest, L2C = db('L2C'), partial = True, mtype = 'RamseyIdxFund', targetvA0 = False,
                    initFromGms = 'init_CRRA_GHH_Static') # use steady state methods to get initial values

### 1.3. Government

Nesting tree:

In [17]:
cesNest = nestingTree.Tree('G', tree = db('nestG').to_list(), f = 'CES')
nest = nestingTree.AggTree(name = 'G', trees = {t.name: t for t in [cesNest]}, ws = ws)
nest(namespace = {str(n)+'_input': n for n in db('n')})

Value shares:

In [18]:
v = nestedShares_noOutputs(nest, ws = ws)
db_vs = v(dbIO)

Add to main database:

In [19]:
db.aom(db_vs('mu').xs(t0), name = 'mu') 
db.aom(adj.rc_pd(db_vs('vD'), nest.get('int').union(nest.get('output'))).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

Add module:

In [20]:
M.addGovernmentModule(nest, L = db('L2C').droplevel('nn').unique())

### 1.4. Simple modules:

In [21]:
# M.addInventoryModule('Itory')
M.addTradeModule('T', partial = True)
M.addEmissionsModule('M', partial = True, mtype = 'EmRegSimpleEOP');
M.compiler.locals['techType'] = "'normal'"
M.addOptWelfareModule('W')
M.addSimulateEVModule('EV')

## 2. Small stuff

*Clean up database a bit (this is not necessary, but it removes some variables that are not ultimately used in the model):*

In [22]:
[db.series.__delitem__(k) for k in ('vD','vTax', 'vD_dur','vD_depr','vD_inv', 'vS', 'pD_dur') if k in db.symbols];

For variables that are defined over $t$, but where we do not yet have an initial value for all $t$, extrapolate from data:

*Note: This forces extrapolation of all variables defined over $t$ - if it is important that some variables are not extrapolated, they should be removed from this statement.*

In [23]:
[symbol.__setattr__('vals', extrapolateUpper(symbol.vals, db('tE')[0])) for symbol in [db[k] for k in db.varDom('t')['t']]];

Merge internally, i.e. write a gdx file from the Python database:

In [24]:
db.mergeInternal()

## 3. Solve and store

Solve model without equilibrium condition:

In [25]:
solvePartial = M.jSolve(10, state = 'C')

Add solution to main database, remove init methods (we want to use this solution as initial value):

In [26]:
[M.db.__setitem__(k, solvePartial[k]) for k in M.db.getTypes(['var']) if k in solvePartial.symbols]; # use solution database
M.db.mergeInternal()
[setattr(m, 'initFromGms', None) for m in M.m.values()];

*Add new modules and resolve*

In [27]:
M.addIndexFundModule('IdxFund', s_idxFund = 's_p', initFromGms = True)
M.addEquilibriumModule('Equi')
fullSol = M.jSolve(10, state = 'C')

Remove final local init methods and add solution to main database:

In [28]:
[M.db.__setitem__(k, fullSol[k]) for k in M.db.getTypes(['var']) if k in fullSol.symbols]; # use solution database
M.db.mergeInternal()
[setattr(m, 'initFromGms', None) for m in M.m.values()];

## 4. Save/export

In [29]:
M.db.data_folder = d['data']

We can save/export the model in a few different ways:
1. Save model instance: The model class ```Model``` (parent class of the CGE class) is pickleable, meaning that we can save/load the class with Python's ```pickle``` class.
2. Store solution: It takes no time to re-initialize the ```Model``` class, so we can also simply store the solution database and - when needed - initialize the model class again.
3. Store GAMS code and solution: This option allows us to remove the model from the python class ```Model``` and instead treat it as a conventional GAMS program.

*Store model instance or database with pickle:*

In [30]:
M.export(repo = M.data_folder, name = M.name) # store the entire class - these are the default options by the way 
M.db.export(repo = M.db.data_folder, name = M.db.name) # store the solution database